In [11]:
import numpy as np
import os
from PIL import Image
from sklearn.decomposition import PCA
from numpy.fft import fft2, fftshift
from sklearn.model_selection import train_test_split
import xgboost as xgb
import random


In [12]:
def random_crop(img, size):
    width, height = img.size   # 기존 이미지의 너비와 높이를 얻습니다.
    new_width, new_height = size
    if width < new_width or height < new_height:
        # 이미지가 충분히 크지 않은 경우에는 사이즈를 늘려줍니다.
        img = img.resize((max(new_width, width), max(new_height, height)))
        width, height = img.size  # 사이즈를 늘린 후의 너비와 높이를 다시 얻습니다.
    
    left = random.randint(0, width - new_width)
    top = random.randint(0, height - new_height)
    right = left + new_width
    bottom = top + new_height

    return img.crop((left, top, right, bottom))

In [13]:
def extract_features(image_path, pca):
    # 이미지를 불러옵니다.
    image = Image.open(image_path).convert('L')
    
    # 이미지를 랜덤 크롭합니다.
    image = random_crop(image, (128, 128))
    
    # 이미지 크기를 재조정합니다.
    image = image.resize((64, 64))
    image = np.array(image)

    # 푸리에 변환을 수행합니다.
    f_transform = fft2(image)
    f_shift = fftshift(f_transform)
    magnitude_spectrum = 20 * np.log(np.abs(f_shift))

    # 평탄화(flatten)하여 1차원 배열로 만듭니다.
    flattened_spectrum = magnitude_spectrum.flatten().reshape(1, -1)

    # PCA를 적용하여 차원을 축소합니다. (여기서는 fit_transform 대신 transform을 사용해야 합니다.)
    pca_features = pca.transform(flattened_spectrum)

    return pca_features.flatten()  # 이제 2차원이 아닌 1차원 배열로 반환합니다.

In [14]:
# PCA 객체 초기화, n_components는 필요한 주성분 수에 따라 조정합니다.
pca = PCA(n_components=50)

# 학습 데이터
train_fake_images = ['FakeImageDetection_Dataset/train/fake_images/' + f for f in os.listdir('FakeImageDetection_Dataset/train/fake_images')]
train_real_images = ['FakeImageDetection_Dataset/train/real_images/' + f for f in os.listdir('FakeImageDetection_Dataset/train/real_images')]

# 검증 데이터
valid_fake_images = ['FakeImageDetection_Dataset/valid/valid_fake_images/' + f for f in os.listdir('FakeImageDetection_Dataset/valid/valid_fake_images')]
valid_real_images = ['FakeImageDetection_Dataset/valid/valid_real_images/' + f for f in os.listdir('FakeImageDetection_Dataset/valid/valid_real_images')]

# 레이블 생성: 가짜 이미지는 1, 진짜 이미지는 0
train_labels = [1] * len(train_fake_images) + [0] * len(train_real_images)
valid_labels = [1] * len(valid_fake_images) + [0] * len(valid_real_images)

# 모든 학습 이미지에 대한 피처를 추출합니다.
train_features = [extract_features(img, pca) for img in train_fake_images + train_real_images]

# 모든 검증 이미지에 대한 피처를 추출합니다.
valid_features = [extract_features(img, pca) for img in valid_fake_images + valid_real_images]


NotFittedError: This PCA instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# DMatrix 형태로 데이터를 변환합니다.
dtrain = xgb.DMatrix(train_features, label=train_labels)
dvalid = xgb.DMatrix(valid_features, label=valid_labels)

# 파라미터 설정
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss'
}

# 학습
bst = xgb.train(params, dtrain, num_boost_round=100, evals=[(dvalid, 'validation')], early_stopping_rounds=10)


In [ ]:
test_images = ['test_data/images/' + f for f in os.listdir('test_data/images')]
test_features = [extract_features(img, pca) for img in test_images]

dtest = xgb.DMatrix(test_features)
predictions = bst.predict(dtest)


In [ ]:
for img, prediction in zip(test_images, predictions):
    print(f'{img}: {"Fake" if prediction < 0.5 else "Real"}')
